In [146]:
import os
import sys
import re
import time
import json
import csv
import glob
import pickle
import pandas as pd
from pathlib import Path
from googletrans import Translator
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize


In [45]:
# scraping dir
scraped_data = '/Users/nyxinsane/Documents/Work - UvA/Automating Equity/Study 1/Study1_Code/scraped_data'

# sites
site_list=['Indeed', 'Glassdoor', 'LinkedIn']

# columns to remove
cols=['Sector', 
      'Sector Code', 
      'Gender', 
      'Age', 
      'Language', 
      'Dutch Requirement', 
      'English Requirement', 
      'Gender_Female', 
      'Gender_Mixed', 
      'Gender_Male', 
      'Age_Older', 
      'Age_Mixed', 
      'Age_Younger', 
      'Gender_Num', 
      'Age_Num', 
      '% Female', 
      '% Male', 
      '% Older', 
      '% Younger']



In [47]:
# This is a manually collected dictionary of incorrect/faulty keywords in scraped site data
with open(f'/Users/nyxinsane/Documents/Work - UvA/Automating Equity/Study 1/Study1_Code/scraped_data/CBS/Data/keyword_trans_dict.txt') as f:
    keyword_trans_dict = json.load(f)

In [48]:
# 111 words to fix
len(keyword_trans_dict)

111

In [49]:
# with open('fix_list.txt', 'w') as f:
#     json.dump(fix_list, f)
    

In [50]:
def fix_broken_linkedin_files(glob_path):
    fix_list = []
    data_dict = {}
    data_list = []

    if glob_path.endswith('.json'):

        with open(glob_path, encoding = 'utf-8') as csv_file_handler:
            csv_reader = csv.DictReader(csv_file_handler)

            for rows in csv_reader:
                first_key = str(list(rows.keys())[0])
                key = rows[first_key]
                data_dict[key] = rows

        for num in data_dict:
            data_list.append(data_dict[num])

        with open(glob_path, 'w', encoding = 'utf-8') as json_file_handler:
            json_file_handler.write(json.dumps(data_list, indent = 4))
    
    return data_list


In [51]:
def fix_keywords(df_temp):
    if len(df_temp) > 0 and isinstance(df_temp, pd.DataFrame):
        for key, value in keyword_trans_dict.items():
            df_temp.loc[
                df_temp['Search Keyword'].astype(str).apply(
                lambda x: x.lower().strip()
                ) == str(key).lower().strip(), 'Search Keyword'
            ] = str(value).lower().strip()

        unfixed = df_temp.loc[
            df_temp['Search Keyword'].astype(str).apply(lambda x: x.lower().strip()).isin([x.lower().strip() for x in list(keyword_trans_dict.keys())])
        ]

        if len(unfixed) != 0:
            for key, value in keyword_trans_dict.items():
                for idx, row in df_temp.iterrows():
                    if row['Search Keyword'].astype(str).lower().strip() == str(key).lower().strip():
                        row.loc[idx, 'Search Keyword'] = str(value).lower().strip()
    

    return df_temp

In [52]:
glob_paths = []

for site in site_list:
    glob_paths.extend(glob.glob(f'{scraped_data}/{site}/Data/*.json')+glob.glob(f'{scraped_data}/{site}/Data/*.csv'))

In [53]:
# 955 json and csv files
len(glob_paths)

955

In [125]:
fix_list = []
appended_data = []

for glob_path in glob_paths[:100]:

    if glob_path.endswith('.json'):
        try:
            df_temp = pd.read_json(glob_path).reset_index()
        except ValueError:
            fix_list.append(glob_path)
            if 'scraped_data/LinkedIn/Data/linkedin_jobs_df_' in glob_path:
                data_json = fix_broken_linkedin_files(glob_path)
                try:
                    df_temp = pd.read_json(glob_path).reset_index()
                except ValueError:
                    fix_list.append(glob_path)
    elif glob_path.endswith('.csv'):
        df_temp = pd.read_csv(glob_path).reset_index()

    if len(df_temp) > 0 and isinstance(df_temp, pd.DataFrame):
        df_temp = fix_keywords(df_temp)
        df_temp.reset_index(drop=True, inplace=True)
        df_temp.drop(columns=cols, axis='columns', inplace=True, errors='ignore')
        df_temp.drop(columns=df_temp.filter(regex=re.compile(r"^unnamed", re.IGNORECASE)).columns, axis='columns', inplace=True, errors='ignore')
        df_temp.drop(columns=df_temp.filter(regex=re.compile(r"^level", re.IGNORECASE)).columns, axis='columns', inplace=True, errors='ignore')
        df_temp.drop(columns=df_temp.filter(regex=re.compile(r"^index", re.IGNORECASE)).columns, axis='columns', inplace=True, errors='ignore')
    
#         if glob_path.endswith('.json'):
#             df_temp.to_json(glob_path, orient='records')
#         elif glob_path.endswith('.csv'):
#             df_temp.to_csv(glob_path, index=False)

        appended_data.append(df_temp.reset_index())

# df_jobs = pd.concat(appended_data)

# if len(df_temp) > 0 and isinstance(df_temp, pd.DataFrame):
#     df_jobs.to_pickle('/Users/nyxinsane/Library/CloudStorage/OneDrive-UvA/Automating Equity/Study 1/Study1_Code/data/df_jobs.pickle')
    
#     df_jobs.to_csv('/Users/nyxinsane/Library/CloudStorage/OneDrive-UvA/Automating Equity/Study 1/Study1_Code/data/df_jobs.csv', index=False)
    



In [126]:
len(fix_list)

0

In [127]:
fix_list

[]

In [128]:
len(appended_data)

8

In [135]:
df_jobs = pd.concat(appended_data)

# if len(df_temp) > 0 and isinstance(df_temp, pd.DataFrame):
#     df_jobs.to_pickle('/Users/nyxinsane/Library/CloudStorage/OneDrive-UvA/Automating Equity/Study 1/Study1_Code/data/df_jobs.pickle')
    
#     df_jobs.to_csv('/Users/nyxinsane/Library/CloudStorage/OneDrive-UvA/Automating Equity/Study 1/Study1_Code/data/df_jobs.csv', index=False)


In [136]:
# len = 9538
len(df_jobs)

9538

In [137]:
df_jobs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9538 entries, 0 to 134
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   index            9538 non-null   int64 
 1   Search Keyword   9538 non-null   object
 2   Platform         9538 non-null   object
 3   Job ID           9538 non-null   object
 4   Job Title        9538 non-null   object
 5   Company Name     9538 non-null   object
 6   Location         9538 non-null   object
 7   Job Description  9538 non-null   object
 8   Rating           9538 non-null   int64 
 9   Employment Type  9538 non-null   int64 
 10  Company URL      9538 non-null   object
 11  Job URL          9538 non-null   object
 12  Job Age          9538 non-null   object
 13  Job Age Number   9538 non-null   object
 14  Collection Date  9538 non-null   object
dtypes: int64(3), object(12)
memory usage: 1.2+ MB


In [138]:
# df_jobs = pd.read_pickle('/Users/nyxinsane/Library/CloudStorage/OneDrive-UvA/Automating Equity/Study 1/Study1_Code/data/df_jobs_dropped.pickle')


In [139]:
df_jobs.columns

Index(['index', 'Search Keyword', 'Platform', 'Job ID', 'Job Title',
       'Company Name', 'Location', 'Job Description', 'Rating',
       'Employment Type', 'Company URL', 'Job URL', 'Job Age',
       'Job Age Number', 'Collection Date'],
      dtype='object')

In [140]:
translator = Translator()
try:
    df_jobs['Language'] = df_jobs['Job Description'].apply(lambda x: translator.detect(str(x).lower().strip()).lang)
except:
    time.sleep(3600)
    df_jobs['Language'] = df_jobs['Job Description'].apply(lambda x: translator.detect(str(x).lower().strip()).lang)
    

ReadTimeout: timed out

In [145]:
import googletrans
dir(googletrans)

['LANGCODES',
 'LANGUAGES',
 'Translator',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'client',
 'constants',
 'gtoken',
 'models',
 'urls',
 'utils']

In [ ]:
df_jobs['Language'].value_counts()

In [ ]:
df_jobs.columns

In [115]:
# Drop duplicates in general and on subset of 'Job Description'
df_jobs.drop_duplicates(inplace=True)
df_jobs.drop_duplicates(subset=['Job Description'], inplace=True)


In [ ]:
# Drop non-english job descriptions
df_jobs.drop(df_jobs.index[df_jobs['Language'] == 'en'], axis='index', inplace=True)


In [ ]:
# 44
len(df_jobs)

In [ ]:
# Find and count unique search keywords
search_keywords = list(set(df_jobs['Search Keyword'].to_list()))


In [ ]:
len(search_keywords)


In [ ]:
search_keywords


In [ ]:
job_descriptions = list(set(df_jobs['Job Description'].to_list()))

In [ ]:
len(job_descriptions)

In [ ]:
job_descriptions

In [ ]:
nltk.download('punkt')


In [ ]:
job_sentences = []
for job_description in job_descriptions:
    job_sentences.append(sent_tokenize(job_description))

In [ ]:
job_sentences